In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/learning_medusa/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [17]:
# hyperparameters
b = 24 # how many independent sequences will we process in parallel?
t = 128 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
lr = 3e-4
eval_iters = 20
d = 128
h = 8
l = 8
dropout = 0.2
l2 = 0.01

medusa_headcount = 3
medusa_discount = torch.tensor(0.8).to(device)

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
v = len(chars)

In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - t - medusa_headcount, (b,))
    x = torch.stack([data[i:i+t] for i in ix])
    y = torch.stack([torch.stack([data[i+1+j:i+t+1+j] for i in ix]) for j in range(medusa_headcount+1)])
    return x.to(device), y.to(device)

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, medusa_logits = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [10]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, 4 * d),
            nn.ReLU(), 
            nn.Linear(4 * d, d),
            nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)

In [11]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(d, head_size, bias=False)
        self.query = nn.Linear(d, head_size, bias=False)
        self.value = nn.Linear(d, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(t, t)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        b,t,d = x.shape
        k = self.key(x)   # (b,t,d/h)
        q = self.query(x) # (b,t,d/h)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (b, t, d/h) @ (b, d/h, t) -> (b, t, t)
        wei = wei.masked_fill(self.tril[:t, :t] == 0, float('-inf')) # (b, t, t)
        wei = F.softmax(wei, dim=-1) # (b, t, t)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (b,t,d/h)
        out = wei @ v # (b, t, t) @ (b, t, d/h) -> (b, t, d/h)
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, h, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(h)])
        self.proj = nn.Linear(head_size * h, d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.dropout(self.proj(torch.cat([head(x) for head in self.heads], dim=-1)))

In [13]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, d, h):
        # d: embedding dimension, h: the number of heads we'd like
        super().__init__()
        head_size = d // h
        self.sa = MultiHeadAttention(h, head_size)
        self.ffwd = FeedFoward(d)
        self.ln1 = nn.LayerNorm(d)
        self.ln2 = nn.LayerNorm(d)

    def forward(self, x):
        return x + self.ffwd(self.ln2(x + self.sa(self.ln1(x))))

In [14]:
class snake(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(d,d)
        self.relu = nn.ReLU() # actual paper uses SiLU bc they build off Llama
        self.w2 = nn.Linear(d,v)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.w2(self.dropout(self.relu(self.w1(x))+x)) # outputs logits shape (b,t,v)

In [25]:
class medusaGPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(v, d)
        self.position_embedding_table = nn.Embedding(t, d)
        self.blocks = nn.Sequential(*[Block(d, h) for _ in range(l)])
        self.ln_f = nn.LayerNorm(d) # final layer norm
        self.lm_head = nn.Linear(d, v)
        
        # Create a list of Medusa heads
        self.medusa_heads = nn.ModuleList([snake() for _ in range(medusa_headcount)])
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, medusa_targets=None, verbose=False):
        b, t = idx.shape
        
        pos_emb = self.position_embedding_table(torch.arange(t, device=device)) # (t,d)
        x = self.ln_f(self.blocks(pos_emb + self.token_embedding_table(idx)))
        
        logits = self.lm_head(x) # (b,t,d)@(d,v)=(b,t,v)
        
        # Apply each snake head to x and store the results
        medusa_logits = torch.stack([head(x) for head in self.medusa_heads], dim=0)
        
        if targets is None:
            loss = None
            medusa_loss = None
        else:
            m, b, t, v = medusa_logits.shape
            logits = logits.view(b*t, v)
            targets0 = targets[0].view(b*t)
            loss = F.cross_entropy(logits, targets0)
            
            medusa_loss = torch.stack([F.cross_entropy(medusa_logits[i].view(b*t, v), targets[i+1].view(b*t)) * medusa_discount**(i+1) for i in range(m)])
            
            loss = loss + medusa_loss.sum()

        return logits, loss, medusa_logits

    def generate_gpt(self, idx, max_new_tokens, temperature=1.0):
        # idx is (b, t) array of indices in the current context
        #assert temperature >= 0
        
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _, __ = self(idx[:, -t:])
            
            # apply softmax to get probabilities
            probs = F.softmax(logits[:, -1, :] / (temperature+1e-10), dim=-1) # (b, d)
            
            idx = torch.cat((idx, torch.multinomial(probs, num_samples=1)), dim=1)
            
        return idx

# Load a saved model

In [26]:
# Assuming `MyModel` is the class of your model
model = medusaGPT().to(device)  # Initialize a model with the same architecture

# Load the saved state dictionary
model.load_state_dict(torch.load('models/medusa_b24_t128_d128_h8_l8_lr0.0003_drop0.2_m3_mdiscount0.8_2024-01-24|17-36-02.pth'))

# If you plan to continue training the model, switch to training mode
#model.train()

# If you only plan to do inference, switch to evaluation mode
model.eval()

medusaGPT(
  (token_embedding_table): Embedding(65, 128)
  (position_embedding_table): Embedding(128, 128)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=128, out_features=16, bias=False)
            (query): Linear(in_features=128, out_features=16, bias=False)
            (value): Linear(in_features=128, out_features=16, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
   

## Regular GPT Inference

for some reason if i try to run this before doing my later functions than the kernel crashes. I recommend restarting with a fresh kernel for those

In [27]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R"
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output = model.generate_gpt(context_tensor, max_new_tokens=500)
print(decode(output[0].tolist()))

JULIET:
O Romeo, Romeo! wherefore art thou Romew my moreor thou, I chinh your do your steecy?


VOLAMP OWISCigifful CLAURENCENCENTIO:
Bow if intern'd inkiltsat thoul thine meed, impe fargelleringmal stren womble blow:
I tooes the mine scomes think as I I, wither, noth age dign umbutiner s,
Bend belied, touch Clerciant as one ones
Of of whe callowal as I plagincheded herself forth us;
We is thee timit co yonague,-----stimbaswoul saided
Throus tharn bon deepourged theirged hunderuchan of wated aloved;
Hearting things thee him the me thouthe
CPU times: user 56.6 s, sys: 2.57 s, total: 59.1 s
Wall time: 58.1 s


In [28]:
print("tokens per second: ", 500/58.1)

tokens per second:  8.605851979345955


# Medusa's first sister Stheno (the aggressive one)

ChatGPT said:
Her name translates to "strength" or "forceful". Stheno was the eldest and most fierce of the sisters, known for her strength and ferocity. 

So I guess since this generation is strictly doing greedy decoding i'll name it after her

In [29]:
def generate_Stheno(model, idx, max_runs):
    assert idx.size(0) == 1, "idx must be of size (1, t)"
    
    logits, loss, mlogits = model(idx[:, -t:])
    mlogits = mlogits[...,-1,:].squeeze(dim=1)
    
    idx_m_prev = torch.argmax(mlogits, dim=-1, keepdim=True).t()
    idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2)
    idx = torch.cat((idx, idx_ntp[:,-1].unsqueeze(dim=0)), dim=1) 

    tok_per_inf = [1]
    
    for _ in range(max_runs-1): 
        
        logits, loss, mlogits = model(torch.cat((idx, idx_m_prev), dim=1)[:, -t:]) 
        idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2) 
        
        match_tensor = (idx_m_prev == idx_ntp[:,-(medusa_headcount+1):-1]).int()
        zero_positions = torch.cat((1 - match_tensor, torch.ones(match_tensor.size(0), 1, dtype=match_tensor.dtype, device=device)), dim=1).argmax(dim=1)
        zero_positions[zero_positions >= match_tensor.size(1)] = match_tensor.size(1)
        range_tensor = torch.arange(match_tensor.size(1), device=device).unsqueeze(0).expand_as(match_tensor)
        mask = range_tensor < zero_positions.unsqueeze(1)
        result = (match_tensor * mask).sum(dim=1).item()

        tok_per_inf.append(result+1)
        
        idx_ntp = idx_ntp[:,-1-medusa_headcount+result].unsqueeze(dim=0)
        idx = torch.cat((idx, idx_m_prev[:,:result], idx_ntp),dim=1)
        
        mlogits = mlogits[...,-1-medusa_headcount+result,:].squeeze(dim=1)
        idx_m_prev = torch.argmax(mlogits, dim=-1, keepdim=True).t()
        
    return idx, tok_per_inf

In [30]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R"
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output, tok_per_inf = generate_Stheno(model, context_tensor, max_runs=250)
print("tokens per inference: ", sum(tok_per_inf)/len(tok_per_inf))
print(decode(output[0].tolist()))

tokens per inference:  1.392
JULIET:
O Romeo, Romeo! wherefore art thou Rome this the shoul the shoul the shame the shame the shame of the soul thine one,
And thaker thally the that I with this the soul of the soul of the soul things one of this soul the shoul the shoul the shame of the soul things one of this soul the shame of the soul things one of this soul the shame of the soul things one of this soul the shoul th
CPU times: user 26.5 s, sys: 1.2 s, total: 27.7 s
Wall time: 27.3 s


In [31]:
print("tokens per second: ", sum(tok_per_inf)/27.3)

tokens per second:  12.747252747252746


#### even tho Stheno is ~45% faster, notice that it's restricted to greedy decoding which means the output quality is lower. If you compare the two passages you'll see that the one above has far less of a problem with repetition

# Medusa's second sister Euryale (the explorative one)

we'll see how this goes



ChatGPT said:

Her name means "far-roaming" in Greek. Euryale was known for her loud crying or bellowing. If your architecture is meant to explore a wide range of possibilities or to "roam" extensively through a dataset, the name Euryale might be suitable. Additionally, if your architecture involves a broad or far-reaching search strategy or is notable for 'broadcasting' its findings extensively (analogous to loud crying), Euryale could be an apt choice.

The goal here with Euryale is effectively to bring probabalistic decoding back to Stheno. My hope is that this will only require more ram and not result in any added latency, but we'll see. The basic idea is to use topk and then randomly or probability-wise select from candidate sequences instead of greedy decoding. If anything i think the re-incorporation of topk results will *maybe* add a speed increase (although likely not as fast as the attention-based mechanism used in actual Medusa)